In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

 Citable  'Colab Notebooks'   mine


In [0]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torchvision.utils import save_image
import os
import matplotlib.pyplot as plt
import numpy
import torch.nn.functional as F
import random
#from utils import *
#from model1 import *


height=256
breadth=256
sample_interval=20
num_epochs =2000
batchsize=4
noise_size=100
clip_value=0.01

cuda = True if torch.cuda.is_available() else False
print(cuda)

True


In [0]:
import torch
from PIL import Image

class Dataset1(torch.utils.data.Dataset):
    def __init__(self,image_path,transform=None):
        self.filename = os.listdir(image_path)
        self.image_path = image_path
        if(transform == None):
            self.transform=transforms.ToTensor()
        else:
            self.transform=transform


    def __getitem__(self,index):
        image=self.transform(Image.open(self.image_path+'/'+self.filename[index]))
        return(image) # C x H x W


    def __len__(self):
        return(len(self.filename))


def ones_target(size):#returns a single dimensional array of ones
    data = Variable(torch.ones(size))
    return data.cuda()

def zeros_target(size):#returns an single dimensional array of zeroes
    data = Variable(torch.zeros(size))
    return data.cuda()

def noise(size):#return a random noise 
    n = Variable(torch.randn(size,3,height,breadth))
    return n.cuda()




def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)#calculating the batch size
    
    optimizer.zero_grad()#initializing the gradients with zero
    
    
    prediction_real = discriminator(real_data)
    #print(prediction_real.size())
    #exit()
    error_real = -torch.mean(prediction_real)#calculating the loss after passing the real images
    error_real.backward()#update the gradients but do not optimize it

    
    prediction_fake = discriminator(fake_data)
    
    error_fake = torch.mean(prediction_fake)#calculating the loss after passing the fake images
    error_fake.backward()
    
    
    optimizer.step()
    
    
    return (error_real + error_fake)*0.5, prediction_real, prediction_fake

def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    
    optimizer.zero_grad()
    
    prediction = discriminator(fake_data)
    error = -torch.mean(prediction)
    error.backward()
    
    optimizer.step()
    
    return error

In [0]:
import torch.nn as nn
class DiscriminatorNet(nn.Module):
    # initializers
    def __init__(self, d=64):
        super(DiscriminatorNet, self).__init__()
        self.conv1 = nn.Conv2d(3, d, 4, 2, 1)    
        self.conv2 = nn.Conv2d(d, d * 2, 4, 2, 1)    
        self.conv2_bn = nn.BatchNorm2d(d * 2)
        self.conv3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d * 4)
        self.conv4 = nn.Conv2d(d * 4, d * 8, 4, 1, 1)
        self.conv4_bn = nn.BatchNorm2d(d * 8)
        self.conv5 = nn.Conv2d(d * 8, 1, 4, 1, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, x):
        #x = torch.cat([input, label], 1)
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = torch.sigmoid(self.conv5(x))
        #print(x.shape)
        return x

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()




class GeneratorNet(nn.Module):
    # initializers
    def __init__(self, d=64):
        super(GeneratorNet, self).__init__()
        # Unet encoder
        self.conv1 = nn.Conv2d(3, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d * 2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d * 2)
        self.conv3 = nn.Conv2d(d * 2, d * 4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d * 4)
        self.conv4 = nn.Conv2d(d * 4, d * 8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d * 8)
        self.conv5 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv5_bn = nn.BatchNorm2d(d * 8)
        self.conv6 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv6_bn = nn.BatchNorm2d(d * 8)
        self.conv7 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        self.conv7_bn = nn.BatchNorm2d(d * 8)
        self.conv8 = nn.Conv2d(d * 8, d * 8, 4, 2, 1)
        # self.conv8_bn = nn.BatchNorm2d(d * 8)

        # Unet decoder
        self.deconv1 = nn.ConvTranspose2d(d * 8, d * 8, 4, 2, 1)
        self.deconv1_bn = nn.BatchNorm2d(d * 8)
        self.deconv2 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d * 8)
        self.deconv3 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d * 8)
        self.deconv4 = nn.ConvTranspose2d(d * 8 * 2, d * 8, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d * 8)
        self.deconv5 = nn.ConvTranspose2d(d * 8 * 2, d * 4, 4, 2, 1)
        self.deconv5_bn = nn.BatchNorm2d(d * 4)
        self.deconv6 = nn.ConvTranspose2d(d * 4 * 2, d * 2, 4, 2, 1)
        self.deconv6_bn = nn.BatchNorm2d(d * 2)
        self.deconv7 = nn.ConvTranspose2d(d * 2 * 2, d, 4, 2, 1)
        self.deconv7_bn = nn.BatchNorm2d(d)
        self.deconv8 = nn.ConvTranspose2d(d * 2, 3, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        e1 = self.conv1(input)
        e2 = self.conv2_bn(self.conv2(F.leaky_relu(e1, 0.2)))
        e3 = self.conv3_bn(self.conv3(F.leaky_relu(e2, 0.2)))
        e4 = self.conv4_bn(self.conv4(F.leaky_relu(e3, 0.2)))
        e5 = self.conv5_bn(self.conv5(F.leaky_relu(e4, 0.2)))
        e6 = self.conv6_bn(self.conv6(F.leaky_relu(e5, 0.2)))
        e7 = self.conv7_bn(self.conv7(F.leaky_relu(e6, 0.2)))
        e8 = self.conv8(F.leaky_relu(e7, 0.2))
        # e8 = self.conv8_bn(self.conv8(F.leaky_relu(e7, 0.2)))
        d1 = F.dropout(self.deconv1_bn(self.deconv1(F.relu(e8))), 0.5, training=True)
        d1 = torch.cat([d1, e7], 1)
        d2 = F.dropout(self.deconv2_bn(self.deconv2(F.relu(d1))), 0.5, training=True)
        d2 = torch.cat([d2, e6], 1)
        d3 = F.dropout(self.deconv3_bn(self.deconv3(F.relu(d2))), 0.5, training=True)
        d3 = torch.cat([d3, e5], 1)
        d4 = self.deconv4_bn(self.deconv4(F.relu(d3)))
        # d4 = F.dropout(self.deconv4_bn(self.deconv4(F.relu(d3))), 0.5)
        d4 = torch.cat([d4, e4], 1)
        d5 = self.deconv5_bn(self.deconv5(F.relu(d4)))
        d5 = torch.cat([d5, e3], 1)
        d6 = self.deconv6_bn(self.deconv6(F.relu(d5)))
        d6 = torch.cat([d6, e2], 1)
        d7 = self.deconv7_bn(self.deconv7(F.relu(d6)))
        d7 = torch.cat([d7, e1], 1)
        d8 = self.deconv8(F.relu(d7))
        o = torch.tanh(d8)

        return o

In [0]:



compose = transforms.Compose(
        [transforms.Resize([height,breadth]),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
data = Dataset1('/content/drive/My Drive/mine/k/M',transform=compose)

data_loader = torch.utils.data.DataLoader(data, batch_size=batchsize, shuffle=True)


discriminator = DiscriminatorNet()
discriminator=discriminator.cuda()

generator=GeneratorNet()
generator=generator.cuda()

d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)#discriminator optimizer
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)#generator optimizer
loss = nn.BCELoss()#Binary Cross Entropy Loss function used
nn=len(data_loader)

In [0]:
print(num_epochs)
e_g=[]
r_e_g=0
e_d=[]
r_d_g=0
e=[]

mdb=0
mgb=0
for epoch in range(num_epochs):
    e.append(epoch)
    r_e_g=0
    r_e_d=0
    for n_batch, data in enumerate(data_loader):
        real_data=data
        real_data=real_data.cuda()
        N = real_data.size(0)
        
        fake_data = generator(noise(N)).detach()

        #train discriminator
        d_error, d_pred_real, d_pred_fake = \
              train_discriminator(d_optimizer, real_data, fake_data)
        r_e_d+=d_error.item()
        #print(r_e_d)
        fake_data = generator(noise(N))
        

        for p in discriminator.parameters():
          p.data.clamp(-clip_value,clip_value)
        
        #train generator
        g_error = train_generator(g_optimizer, fake_data)
        r_e_g+=g_error.item()
        #print(r_e_g)
        
        
        batches_done = epoch * len(data_loader) + n_batch
        if(epoch>=20 and batches_done % sample_interval == 0):
            batch_done="epoch "+str(epoch)+"n_batch "+str(n_batch)
            save_image(fake_data.data, '/content/drive/My Drive/mine/M/model3/images/%s.png' % batch_done, nrow=5, normalize=True)
   
    if(epoch == 0 or mdb>r_e_d):
        mdb=r_e_d
        torch.save(discriminator.state_dict(),"/content/drive/My Drive/mine/M/model3/discriminatorM.pth")
    
    
    if(epoch ==0 or mgb>r_e_g):
        torch.save(generator.state_dict(),"/content/drive/My Drive/mine/M/model3/generatorM.pth")
      
    print("epoch end------------------------------"+str(epoch))
    e_g.append(r_e_g/nn)
    e_d.append(r_e_d/nn)
    
    plt.figure(1)
    plt.plot(e,e_g,color='blue')
    plt.plot(e,e_d,color='green')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('GAN')
    plt.savefig("/content/drive/My Drive/mine/M/model3/trainlossM.png")
    
    
    #plt.figure(2)
    #plt.savefig("/content/drive/My Drive/mine/M/model/generatorgrad.png")

torch.save(generator.state_dict(),"/content/drive/My Drive/mine/M/model3/generatorML.pth")

2000
epoch end------------------------------0
epoch end------------------------------1
epoch end------------------------------2
epoch end------------------------------3
epoch end------------------------------4
epoch end------------------------------5
epoch end------------------------------6
epoch end------------------------------7
epoch end------------------------------8
epoch end------------------------------9
epoch end------------------------------10
epoch end------------------------------11
epoch end------------------------------12
epoch end------------------------------13
epoch end------------------------------14
epoch end------------------------------15
epoch end------------------------------16
epoch end------------------------------17
epoch end------------------------------18
epoch end------------------------------19
epoch end------------------------------20
epoch end------------------------------21
epoch end------------------------------22
epoch end------------------------------